## Using LLM Model in Langchain

In [7]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

model = ChatGroq(
    model="openai/gpt-oss-20b",
    temperature=0
)

response = model.invoke("Tell me a joke")
# print(response)
print(response.content)

Why don’t skeletons fight each other?

They don’t have the guts.


In [ ]:
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage, SystemMessage

from dotenv import load_dotenv

load_dotenv()


model = ChatGroq(
    model="openai/gpt-oss-20b",
    temperature=0
)

system_msg = SystemMessage(
    "You are a helpful assistant that responds to questions with three exclamation marks."
)
human_msg = HumanMessage("What is the capital of France?")

response = model.invoke([system_msg, human_msg])
# print(response)
print(response.content)

`HumanMessage`
A message sent from the perspective of the human, with the user role

`AIMessage`
A message sent from the perspective of the AI that the human is interacting with, with the assistant role

`SystemMessage`
A message setting the instructions the AI should follow, with the system role

`ChatMessage`
A message allowing for arbitrary setting of role

In [10]:
from langchain_core.prompts import PromptTemplate

template = PromptTemplate.from_template("""Answer the question based on the
    context below. If the question cannot be answered using the information 
    provided, answer with "I don't know".

Context: {context}

Question: {question}

Answer: """)

template.invoke({
    "context": """The most recent advancements in NLP are being driven by Large 
        Language Models (LLMs). These models outperform their smaller 
        counterparts and have become invaluable for developers who are creating 
        applications with NLP capabilities. Developers can tap into these 
        models through Hugging Face's `transformers` library, or by utilizing 
        OpenAI and Cohere's offerings through the `openai` and `cohere` 
        libraries, respectively.""",
    "question": "Which model providers offer LLMs?"
})

StringPromptValue(text='Answer the question based on the\n    context below. If the question cannot be answered using the information \n    provided, answer with "I don\'t know".\n\nContext: The most recent advancements in NLP are being driven by Large \n        Language Models (LLMs). These models outperform their smaller \n        counterparts and have become invaluable for developers who are creating \n        applications with NLP capabilities. Developers can tap into these \n        models through Hugging Face\'s `transformers` library, or by utilizing \n        OpenAI and Cohere\'s offerings through the `openai` and `cohere` \n        libraries, respectively.\n\nQuestion: Which model providers offer LLMs?\n\nAnswer: ')

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
template = ChatPromptTemplate.from_messages([
    ('system', '''Answer the question based on the context below. If the 
        question cannot be answered using the information provided, answer with 
        "I don\'t know".'''),
    ('human', 'Context: {context}'),
    ('human', 'Question: {question}'),
])

template.invoke({
    "context": """The most recent advancements in NLP are being driven by Large 
        Language Models (LLMs). These models outperform their smaller 
        counterparts and have become invaluable for developers who are creating 
        applications with NLP capabilities. Developers can tap into these 
        models through Hugging Face's `transformers` library, or by utilizing 
        OpenAI and Cohere's offerings through the `openai` and `cohere` 
        libraries, respectively.""",
    "question": "Which model providers offer LLMs?"
})

Prompt community

https://smith.langchain.com/hub/

In [ ]:
# Create a LANGSMITH_API_KEY in Settings > API Keys
from langsmith import Client

from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

client = Client(api_key=os.environ.get("LANGSMITH_API_KEY"))
prompt = client.pull_prompt("hardkothari/prompt-maker", include_model=True)

## Specific Formats out of LLMs

In [12]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

from pydantic import BaseModel

class AnswerWithJustification(BaseModel):
    '''An answer to the user's question along with justification for the 
        answer.'''
    answer: str
    '''The answer to the user's question'''
    justification: str
    '''Justification for the answer'''

model = ChatGroq(
    model="openai/gpt-oss-20b",
    temperature=0
)

structured_llm = model.with_structured_output(AnswerWithJustification)

structured_llm.invoke("""What weighs more, a pound of bricks or a pound 
    of feathers""")

AnswerWithJustification(answer='They weigh the same – both are one pound.', justification='A pound is a unit of mass, so a pound of bricks and a pound of feathers each have the same mass of one pound. The difference is in volume and density, not weight.')

## Runnable Interface
There is a common interface with these methods:

`invoke`: transforms a single input into an output

`batch`: efficiently transforms multiple inputs into multiple outputs

`stream`: streams output from a single input as it’s produced

In [18]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

model = ChatGroq(
    model="openai/gpt-oss-20b",
    temperature=0
)

completion = model.invoke('Hi there!') 
# Hi!

# completions = model.batch(['Hi there!', 'Bye!'])
# # ['Hi!', 'See you!']

# for token in model.stream('Bye!'):
#     print(token.content)
#     # Good
#     # bye
#     # !

completion.content

'Hello! 👋 How can I help you today?'

In [19]:
completions = model.batch(['Hi there!', 'Bye!'])

completion.content

'Hello! 👋 How can I help you today?'

In [ ]:
for token in model.stream('Tell me a joke!'):
    # IMPORTANT: Keep the processing of each chunk as efficient as possible.
    # While you're processing the current chunk, the upstream component is
    # waiting to produce the next one. For example, if working with LangGraph,
    # graph execution is paused while the current chunk is being processed.
    # In extreme cases, this could even result in timeouts (e.g., when llm outputs are
    # streamed from an API that has a timeout).
    print(token.content)
    # Good
    # bye
    # !
















Why
 don
’t
 skeleton
s
 fight
 each
 other
?


They
 don
’t
 have
 the
 guts
!



In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

model = ChatGroq(
    model="openai/gpt-oss-20b",
    temperature=0
)

prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
parser = StrOutputParser()
chain = prompt | model | parser

async for event in chain.astream_events({"topic": "parrot"}):
    kind = event["event"]
    if kind == "on_chat_model_stream":
        # Ambil content dari chunk
        content = event["data"]["chunk"].content
        if content:
            print(content, end="", flush=True)

Why did the parrot bring did the parrot bring a ladder to the bar?

 a ladder to the bar?

Because it heard the drinksBecause it heard the drinks were on the house! 🦜 were on the house! 🦜🍹🍹